In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path
import zipfile
import pandas as pd
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn import set_config
from sklearn.preprocessing import RobustScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns

# Create API connection to Kaggle

In [ ]:
# Create kaggle directory
os.makedirs(os.path.expanduser("~/.kaggle"),exist_ok=True)
# Set permissions(required by kaggle api)
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"),0o600)

## Test the connection to Kaggle before loading the data

In [ ]:
api = KaggleApi()
api.authenticate()

# Test with a simple call
datasets = api.dataset_list(search='titanic')
for d in datasets[:3]:
    print(d.ref, "-", d.title)

# Create a method that downloads and loads the data into a dataframe

In [ ]:
def load_housing_data():
    dataset_path=Path("/Users/luthfi/sandbox/Kaggle_competitions/Machine_learning/textbook/sandbox/datasets/Real%20estate.csv")
    Path("datasets").mkdir(parents=True,exist_ok=True)

    if not dataset_path.exists():
        # Authenticate with kaggle api
        api = KaggleApi()
        api.authenticate()

        # Download dataset
        api.dataset_download_file(
            dataset="quantbruce/real-estate-price-prediction",
            file_name="Real estate.csv",
            path="datasets",
            force=True
        )

        # Open the file in the local folder created
        try:
            with open("/Users/luthfi/sandbox/Kaggle_competitions/Machine_learning/textbook/sandbox/datasets/Real%20estate.csv","r") as file:
                content= file.read()
        except FileNotFoundError:
            print("File doesn't exist locally")

    return pd.read_csv(dataset_path)

In [ ]:
df = load_housing_data()
df.head()

In [ ]:
df.shape

# Check for any missing values

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.hist(bins=50,figsize=(12,8))

In [ ]:
df.columns

# Split the data

In [ ]:
train_set,test_set = train_test_split(df,random_state=42,test_size=0.2)

In [ ]:
train_set.shape

In [ ]:
test_set.shape

# Explore + Visualize the data to gain insights - manually

## Manual Changes

### Change the name so is uniform

In [ ]:
manual_train = train_set.copy()

In [ ]:
manual_train.head()

In [ ]:
manual_train.reset_index(drop=True,inplace=True)
manual_train= manual_train.drop(columns='No')

In [ ]:
names = list(manual_train.columns)
names

In [ ]:
for i in range(len(names)):
    names[i] = names[i][2:] # This removes the X1
    names[i] = names[i].lstrip() # Removes the from space
print(names)

In [ ]:
manual_train.columns = names
manual_train.head()

### Transaction date - change so its only contains just the year

In [ ]:
manual_train['transaction date'].dtype

In [ ]:
manual_train['transaction date'] = pd.to_datetime(
    manual_train['transaction date'].astype(int), format='%Y'
).dt.year

In [ ]:
manual_train.dtypes

In [ ]:
manual_train['transaction date']

### Visualize geographical data

In [ ]:
manual_train.plot(kind= "scatter",x="longitude",y="latitude",grid=True)
plt.show()

Density of points:
- When many points overlap, their transparency adds up visually — the more overlap, the darker that region becomes.
- This creates a map of where the houses are. With alpha=0.2:

    Darker areas = more houses listed there

    Lighter/sparse areas = fewer houses
- alpha=0.2 helps you:

    Spot clusters or hot spots where houses are densely located

    Avoid the problem of "overplotting" (when points sit on top of each other and hide information)

    Make better visual decisions about patterns in the data

#### Density of points

In [ ]:
manual_train.plot(kind="scatter",x="longitude",y="latitude",grid=True,alpha=0.2)
plt.show()

### Look for correlations

In [ ]:
corr_matric = manual_train.corr()
corr_matric['house price of unit area'].sort_values(ascending=False)

### Transformers

Transform data to handle skewed data + Reduce the impace of outliers + Make the distribution normal

Plot the distribution to see its skewness, outliers, or non-normality

- Histogram : Shows the shape (skew,normal,uniform)
- Box plot : To detect outliers (shows median,IQR, and outliers clearly)

✅ Use StandardScaler when:

- You're using models that assume normality or optimize on gradients (e.g., linear models, logistic regression, SVM, PCA)

- Your data is approximately normal

- You want values centered around 0 and scaled by standard deviation

✅ Use MinMaxScaler when:

- You need all features to be in a specific range, like [0, 1] (often for neural networks)

- Your data is not normal, but you still need scaling

- You're visualizing or applying distance-based models where range matters

In [ ]:
manual_train.columns

#### transaction date

In [ ]:
manual_train['transaction date']

In [ ]:
# Histogram

sns.histplot(manual_train['transaction date'], kde=True)
plt.title('Histogram of transaction date')
plt.show()

Histogram interpretation :
- One spike at 2012.0 and another at 2013.0
- These nothing in the middle - it looks bimodal and almost like a categorical split between two years
- This shows the dates aren't continuous over time - maybe there are transactions only at the start and end of a period.

In [ ]:
# Box and whisker

manual_train.boxplot(column='transaction date')
plt.title('Box and whisker of tranasaction date')
plt.show()

Box plot interpreatation :
- The box is very tall, stretching from 2012 to 2013, and theres very little detail in the quartiles, suggesting only two main groups of dates.
- It confirms that theres no smooth distribution, just a jump from one group of dates to another.

Transformer : 

- No transformers are used for dates in general because its a time based variable.
- Dates are not true numerical features — they're temporal, meaning their value is only meaningful in the context of time.

#### house age

In [ ]:
sns.histplot(manual_train['house age'],kde=True)
plt.title('Histogram of house age')
plt.show()

Histrogram interpretation :
- Distribution is midly skewed to the righ
- Most values are concetrated between 0 - 20, but there is a small bump around 30 - 40
- No extreme outliers

In [ ]:
# Box and whisker

manual_train.boxplot(column='house age')
plt.title('Box and whisker for house age')
plt.show()

Box and whisker :
- The data is slightly skewed to the right and the tail is longer on the right.
- The medium is not exactly in the middle but more to the left.
- No outliers can be seen

Transformer :
No transformer is needed. As the data is more or less uniform.

#### distance to the nearest MRT station

In [ ]:
# Histogram

sns.histplot(manual_train["distance to the nearest MRT station"],kde=True)
plt.title("Histogram for distance to the nearest MRT station")
plt.show()

Applying the log trasnformation to see the difference

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
manual_train["distance to the nearest MRT station"].hist(ax=axs[0],bins=50)
manual_train["distance to the nearest MRT station"].apply(np.log).hist(ax=axs[1],bins=50)
axs[0].set_xlabel("distance to the nearest MRT station")
axs[1].set_xlabel("log of the distance to the nearest MRT station")
axs[0].set_ylabel("Number of station")
plt.show()

Histogram interpretation :

- The histogram clearly shows a right-skewed distribution. A large number of observations are clustered at lower distances (0-1000), and the frequency gradually decreases as the distance increases.

- There's a significant tail extending towards higher distances, indicating some properties are much further from an MRT station than the majority.

- The presence of the Kernel Density Estimate (KDE) curve further confirms this skewed shape.

In [ ]:
# Box and whisker

manual_train.boxplot(column="distance to the nearest MRT station")
plt.title("Box and whisker for distance to the nearest MRT station")
plt.show()

Box and whisker intrepretation :
- The data is highly skewed to the right 
- There are many outliers
- The median is more to the left of the box

Transformer : 
Given the rightly skewed data + outliers log transformer.

#### number of convience stores

In [ ]:
# Histogram

sns.histplot(manual_train["number of convenience stores"],kde=True)
plt.title("Histogram for number of convenience stores")
plt.show()

Histogram interpretation :
- There is a high count at 0, a dip, then another peak around 4-6, and then another peak at 6, followed by general decrease.
- Counts generally decrease after the peak around 6, though there are still significant counts up to 10.
- This is not a clear bell curve. Theres a noticeable frequency of properties with 0 convience stores. The distribution extends up to 10 convience stores.

A bimodal distribution is a probability distribution with two distinct peaks (or "humps") in its histogram or density plot. You’ll see this as two visible "hills" or clusters in the data when plotted.

- Unimodal = 1 peak
- Bimodal = 2 peaks
- Multimodal = 3 or more peaks

Think of "modal" as “mode” = the most frequent value or region.
So multimodal means there are multiple areas where the data tends to cluster.

In [ ]:
# box and whisker

manual_train.boxplot(column="number of convenience stores")
plt.title("Box and whisker for number of convenience stores")
plt.show()

Box and whisker intrepertation :
-  The upper whisker extends all the way to 10, and the lower whisker extends to 0. This indicates that the data is not significantly skewed in the conventional sense, especially not "rightly skewed."
- No outliers
- the median is 4. 

Transformer : 

- The histogram shows multiple peaks, suggesting a somewhat uniform or possibly bimodal distribution rather than a strongly skewed one.
- Given that the data is not significantly skewed and covers a discrete, limited range (0 to 10), a non-linear transformer (like a log transformer) is generally not needed or recommended.
- Log transformations are used to make highly skewed data more symmetrical, which isn't the primary characteristic here. Applying a log transform to data that includes 0 and is not heavily skewed can sometimes be more problematic than beneficial.

- No transformers

#### Latitude and longitude

- If no clustering or distance based tasks then apply only standardization and no custom transformer is needed.

- If you are doing clustering KNN or distance-based , don't standardize but rather use a custiner transformer to compute real geographical distances if needed.

### Feature Scaling

#### transaction date

No — you'd use standardization only if the extracted date-based feature is continuous and meaningful for that model (like time intervals). Not because it's a date.

#### house age

Standardization would be best because the data is roughly normal.

#### distance to the nearest MRT station

Feature Scaler:

- Robust Scaler (sklearn.preprocessing.RobustScaler): After applying a logarithmic transformation, the distribution will be less skewed, but outliers might still have a disproportionate effect on standard scaling methods (like StandardScaler).

- RobustScaler scales features using statistics that are robust to outliers. It scales the data according to the Interquartile Range (IQR) and median, which are less affected by extreme values than the mean and standard deviation used by StandardScaler. This is particularly beneficial given the clear outliers observed in your boxplot, even after a potential log transformation.

#### number of convience stores

- Standard Scaler

#### Latitude and longitude

- If no clustering or distance based tasks then apply only standardization and no custom transformer is needed.

- If you are doing clustering KNN or distance-based , don't standardize but rather use a custiner transformer to compute real geographical distances if needed.

### Scaling the target

- Its better when all features are on the same scale / range
- Works best for linear regression, gradient descent, etc
- Helps the model train better - learns is mde easiser for the model

In [ ]:
target_variable = manual_train["house price of unit area"]
target_variable.describe()

In [ ]:
# histogram

sns.histplot(target_variable,kde=True)
plt.title("Histogram for the target variable")
plt.show()

In [ ]:
target_variable.skew()

| Skew Value   | Interpretation      | What to Do                               |
| ------------ | ------------------- | ---------------------------------------- |
| 0.0          | Perfectly symmetric | No transformation needed                 |
| 0.0 to ±0.5  | Slight skew         | Scaling usually enough                   |
| ±0.5 to ±1.0 | **Moderate skew**   | Consider `StandardScaler`, maybe try log |
| > ±1.0       | Strong skew         | Use `log1p()` or another transformation  |


In [ ]:
model_manual_train = TransformedTargetRegressor(LinearRegression(),transformer=StandardScaler())

In [ ]:
model_manual_train.fit(manual_train[["house price of unit area"]],target_variable)

In [ ]:
some_new_data = manual_train[["house price of unit area"]].iloc[:5]
print(some_new_data)

In [ ]:
predictions = model_manual_train.predict(some_new_data)
print(predictions)

## Changes using the pipeline

The purpose of fit() and transform():

- fit() : learns what must do to the data like the changes -everything
- transform() : applys the changes that was done in the fit()
- __init__() : only use when you're passing any external parameters

In [ ]:
pipe_train = train_set.copy()

In [ ]:
pips = pipe_train.drop(columns=["Y house price of unit area"])

In [ ]:
pips

### Change all column names into a uniform

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class Column_name_transformer(BaseEstimator,TransformerMixin):

    def fit(self, X, y=None):
        self.new_columns = [col[2:].lstrip() for col in X.columns]
        return self

    def transform(self, X):
        x_copy = X.copy()
        x_copy.columns = self.new_columns
        return x_copy

    def get_feature_names_out(self, input_features=None):
        return self.new_columns

In [ ]:
#change_name = Column_name_transformer()
#pipe_train = change_name.fit_transform(pipe_train)
#pipe_train

### Transaction date - change to only contain the year

In [ ]:
class transaction_date_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.column_name] = X[self.column_name].astype(int).round()
        return X

    def get_feature_names_out(self, input_features=None):
        # Return the same column name after transformation
        return [self.column_name]

In [ ]:
#transaction_date_transform = transaction_date_transformer(column_name="transaction date")
#pipe_train['transaction date'] = transaction_date_transform.fit_transform(pipe_train)
#pipe_train

### drop columns that is not needed

In [ ]:
class drop_columns_not_needed(BaseEstimator,TransformerMixin):

    def __init__(self,number_columns):
        self.number_columns=number_columns

    def fit(self,X,y=None):
        X.reset_index(drop=True,inplace=True)
        self.new_x = X.iloc[:,self.number_columns:]
        return self
    
    def transform(self,X):
        return self.new_x

In [ ]:
#dropping_index_columns = drop_columns_not_needed(number_columns=1)
#pipe_train = dropping_index_columns.fit_transform(pipe_train)
#pipe_train.head()

## Preprocessing pipeline

In [ ]:
log_pipeline=make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log,feature_names_out="one-to-one"),
    RobustScaler()
)

In [ ]:
default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())

Call all the custom functions

In [ ]:
change_name = Column_name_transformer()
transaction_date_transform = transaction_date_transformer(column_name="X1 transaction date")
dropping_index_columns = drop_columns_not_needed(number_columns=1)

In [ ]:
#test_pipe = ColumnTransformer([
#    ("log", log_pipeline, ["distance to the nearest MRT station", "house price of unit area"])
#], remainder='passthrough')

#test_pipe.fit_transform(pipe_train)

In [ ]:
#transaction_date = transaction_date_transformer(column_name="transaction date")

#test_pipe = ColumnTransformer([
#    ("transaction_date_transform", transaction_date, ["transaction date"])
#], remainder="passthrough")

#test_pipe.fit_transform(pipe_train)

In [ ]:
#dropper = drop_columns_not_needed(number_columns=1)

#test_pipe = ColumnTransformer([
#    ("drop_index", dropper, pipe_train.columns.tolist())  # or just ["column1", "column2", ...]
#], remainder="passthrough")

#test_pipe.fit_transform(pipe_train)

## Apply the preprocessing pipeline

In [ ]:
# Step 1: Drop the 'No' column
X = pipe_train.drop(columns=["No", "Y house price of unit area"])
y = pipe_train["Y house price of unit area"]

In [ ]:
# Step 2: Use updated pipeline
final_pipeline = make_pipeline(
    Column_name_transformer(),
    transaction_date_transformer("transaction date"),
    ColumnTransformer([
        ("log", log_pipeline, ["distance to the nearest MRT station"]),
    ], remainder=default_num_pipeline),
    LinearRegression()
)

## Train the model

In [ ]:
# Step 3: Train model
final_pipeline.fit(X, y)

In [ ]:
# Step 4: Predict
housing_predictions = final_pipeline.predict(X)

In [ ]:
# Step 5: Check first 5 predictions
print("Predictions:", housing_predictions[:5].round(1))
print("True values:", y.iloc[:5].values)

In [ ]:
# Step 6: Relative Error
error_ratios = housing_predictions[:5] / y.iloc[:5].values - 1
print("Errors:", ", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

In [ ]:
# Step 7: RMSE
lin_rmse = np.sqrt(mean_squared_error(y, housing_predictions))
print("Linear Regression RMSE:", round(lin_rmse, 2))

## Data after it went through the pipeline

In [ ]:
# Fit the preprocessing pipeline (exclude model step)
preprocessor_only = make_pipeline(
    Column_name_transformer(),
    transaction_date_transformer("transaction date"),
    ColumnTransformer([
        ("log", log_pipeline, ["distance to the nearest MRT station"]),
    ], remainder=default_num_pipeline)
)

In [ ]:
# Fit on X
preprocessor_only.fit(X)

In [ ]:
# Transform X
X_prepared = preprocessor_only.transform(X)

In [ ]:
# Get feature names
column_transformer = preprocessor_only.named_steps["columntransformer"]
feature_names = column_transformer.get_feature_names_out()

In [ ]:
# Put into DataFrame
X_prepared_df = pd.DataFrame(X_prepared, columns=feature_names)

print("After preprocessing:")
print(X_prepared_df.head())